In [31]:
# import os 
# from langchain_teddynote.messages import stream_response
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pandas as pd


from utils.RAGPipeLine import Ragpipeline


In [32]:
pipeline = Ragpipeline()

[초기화] vector_store 초기화 완료
[초기화] retriever 초기화 완료
[초기화] RAG chain 초기화 완료
[초기화] RAG title chain 초기화 완료
[초기화] RAG post chain 초기화 완료


In [33]:
vectorDB = pipeline.init_vectorDB()

[초기화] vector_store 초기화 완료


### 1. LLM 모델 준비 

In [19]:
from langchain_community.chat_models import ChatOllama
from langchain_community.embeddings.ollama import OllamaEmbeddings

from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate

from utils.config import config

In [20]:
gemma2_llm =  ChatOllama(model="gemma2")
qwen2_llm = ChatOllama(model="qwen2")
our_llm = ChatOpenAI(
            model       = config['llm_predictor']['model_name'],
            temperature = config['llm_predictor']['temperature'],
            # streaming=True,
            # callbacks=[StreamCallback()]
        )

gemma2_embeddings = OllamaEmbeddings(model="gemma2")

In [21]:
retriever = pipeline.init_retriever()
mq_retriever = pipeline.init_multi_query_retriever()
sq_retriever = pipeline.init_self_query_retriever()
pd_retriever  = pipeline.init_parent_document_retriever()
web_retriever = pipeline.init_web_research_retriever()

[초기화] retriever 초기화 완료


In [22]:
from utils.prompt import contextualize_q_prompt   
from utils.prompt import qa_prompt2                  # 게시물 생성 프롬프트 
from utils.prompt import title_generator_prompt     # 제목 생성 프롬프트 
from utils.prompt import text_generator_prompt      # 게시물 생성 프롬프트 

In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain


In [24]:
def set_chain(llm, retriever, prompt):
    
    question_answer_chain = create_stuff_documents_chain(llm, prompt)           # 문서 목록을 가져와서 모두 프롬프트로 포맷한 다음 해당 프롬프트를 LLM에 전달합니다.
    
    rag_chat_chain = create_retrieval_chain(retriever, question_answer_chain)  # 사용자 문의를 받아 리트리버로 전달하여 관련 문서를 가져옵니다. 그런 다음 해당 문서(및 원본 입력)는 LLM으로 전달되어 응답을 생성

    
    return rag_chat_chain

In [25]:
query = '인도 통관 및 운송'

docs = retriever.get_relevant_documents(query)

docs

[Document(page_content='제Ⅲ절\n인도의 통관 절차\n01\n통관 조직 및 절차\n', metadata={'category': '3. 정책 및 무역', 'datetimes': '2024-07-08 16:42:32', 'filename': 'data/PDF_with_contents/3. 정책 및 무역/[정책][관세청][2019.11.07]신남방국 통관·통상환경 및 FTA 활용방안.pdf', 'page_no': 9, 'subcategory': '1. 통관 조직 및 절차'}),
 Document(page_content='인도 전문 식품수입 벤더현황\n다. 통관 및 검역 유의사항\n(1) 인도 통관, 사전 준비가 관건\n▪\n인도의 통관은 수입 통관 및 검역을 위한 사전 준비 절차가 중요함\n▪\n통관에 필요한 서류를 제출하면 서류의 기재 오류나 서류 상호간 불일치를\n수정하기가 매우 어려우며 이로 인해 통관 절차가 2~3주씩 지연되는 경우도\n있어 철저한 서류 작성이 필수적임\n▪\n또한 인도로 수출하는 물품의 경우 포장명세서(P/L)와 상업송장(C/I)을\n컨테이너별로 철저히 작성해야 함. 인도 세관은 한 컨테이너 안에서 박스별로\n화물의 일치 여부를 조사하는 등 실물 검사가 철저하며 서류와 화물 불일치\n시 통관이 불가능함\n▪\n특히 인도는 도로와 항만 등 인프라가 낙후되어 있고 운송수단도 노후하기\n때문에 내륙운송 과정에서 사고가 자주 발생하므로 내륙운송보험에 가입해야\n함\n43\n', metadata={'category': '3. 정책 및 무역', 'datetimes': '2024-07-08 16:41:51', 'filename': 'data/PDF_with_contents/3. 정책 및 무역/[정책][KATI 농식품수출정보][2019]인도 전문 식품수입.pdf', 'page_no': 21, 'subcategory': '다. 통관 및 검역 유의사항'}),
 Document(page_content='5 통관 및 운송\n가 통관제도\n통관 

In [26]:
# Chain 실행
rag_chain = set_chain(our_llm, retriever, qa_prompt2)
question = "인도 펫케어 식품 시장 동향을 알려주세요."
response = rag_chain.invoke({'input':question})
print(response)

{'input': '인도 펫케어 식품 시장 동향을 알려주세요.', 'context': [Document(page_content='24 6 27 오후 310 emericsorg446aifcommonPrintPopupesbrdctsNo354483systemcode02mida30400000000\n인도 펫케어 식품 시장 동향\n인도남아시아 인도 20231010\n인도 반료동물 사료 시장은 연간 15 성장 중\n반려동물 제품의 고급화에 따라 제품의 세분화 맞춤형 니즈가 증가할 것으로 예상\n인도 펫케어 산업은 지속 성장 중\n시장조사 전문 기관 유로모니터 펫 케어 산업 조사에 따르면 2023년 전세계 펫 케어 시장 규모는 약 1천 844억', metadata={'chunk': 0, 'filename': 'data/PDF_without_contents\\2. 경제 및 시장 분석\\펫산업\\[인도_펫산업_시장분석][AIF][2023-10-10]인도 펫케어 식품 시장 동향.pdf', 'keyword': '키워드 1: 인도\n키워드 2: 펫케어 식품 시장\n키워드 3: 동향'}), Document(page_content='인도 펫푸드 시장의 경쟁 동향\n인도 펫푸드 시장은 주요 대기업들이 장악하고 있으며 이들은 신제품 개발에\n적극적으로 투자하고 있다 인수 및 확장을 통해 사업 영역을 넓히는 중이며\n연구와 개발에 집중하여 보다 저렴한 가격의 새로운 제품을 시장에 선보이고\n있다\n인도 펫푸드 시장의 최근 동향\n인도의 유명 제약 회사인 Mankind Pharma 는 인도에서 반려동물을 키우는\n인구가 증가함에 따라 펫푸드 시장 진출을 결정하고 최근 자체 브랜드 Petstar를', metadata={'chunk': 6, 'filename': 'data/PDF_without_contents\\2. 경제 및 시장 분석\\펫산업\\[인도_펫산업_시장분석][강원 수출기업 서포트][2024-04-14]인구 14억 인도의 반려동물 사랑 펫푸드 시장도 동반 성장 중.pdf', 'keyw

In [27]:
print(response['answer'])
print('---'*10)
for con in response['context']:
    print(con.metadata)

인도 펫케어 식품 시장은 현재 지속적인 성장을 보이고 있습니다. 반려동물 제품의 고급화로 인해 제품의 세분화와 맞춤형 니즈가 증가하고 있으며, 이는 펫케어 산업의 성장을 이끌고 있습니다. 유로모니터에 따르면 2023년 전세계 펫 케어 시장의 규모는 약 1천 844억이며, 이 중 인도의 펫케어 시장도 큰 부분을 차지하고 있습니다. 또한, 인도의 유명 제약 회사인 Mankind Pharma가 펫푸드 시장에 진출하고 있으며, 이는 반려동물을 키우는 인구의 증가에 대한 대응으로 볼 수 있습니다. 이러한 동향들을 통해 인도 펫케어 식품 시장은 더 많은 기회와 성장 잠재력을 보유하고 있습니다.
------------------------------
{'chunk': 0, 'filename': 'data/PDF_without_contents\\2. 경제 및 시장 분석\\펫산업\\[인도_펫산업_시장분석][AIF][2023-10-10]인도 펫케어 식품 시장 동향.pdf', 'keyword': '키워드 1: 인도\n키워드 2: 펫케어 식품 시장\n키워드 3: 동향'}
{'chunk': 6, 'filename': 'data/PDF_without_contents\\2. 경제 및 시장 분석\\펫산업\\[인도_펫산업_시장분석][강원 수출기업 서포트][2024-04-14]인구 14억 인도의 반려동물 사랑 펫푸드 시장도 동반 성장 중.pdf', 'keyword': '키워드: 인도, 펫푸드 시장, 경쟁 동향, 대기업, 신제품 개발, 인수, 확장, 연구, 개발, 저렴한 가격, 새로운 제품, Mankind Pharma, Petstar, 반려동물 인구 증가.'}
{'chunk': 4, 'filename': 'data/PDF_without_contents\\2. 경제 및 시장 분석\\펫산업\\[인도_펫산업_시장분석][AIF][2023-10-10]인도 펫케어 식품 시장 동향.pdf', 'keyword': '키워드 1: 반려동물 친화 공간\n키워드 2: 사료 시장 성장\n키워드 3: 프리미엄 펫케어 

In [28]:
# Chain 실행
rag_chain = set_chain(our_llm, mq_retriever, qa_prompt2)
question = "인도 펫케어 식품 시장 동향을 알려주세요."
response = rag_chain.invoke({'input':question})
print(response['answer'])

1. 인도 펫케어 식품 시장에 대한 최신 동향은 무엇인가요?
2. 인도의 펫케어 식품 시장 동향을 알고 싶습니다. 정보를 제공해 주세요.
3. 인도에서 펫케어 식품 시장이 어떻게 변화하고 있는지 알려주세요.인도 펫케어 식품 시장은 현재 지속적인 성장을 보이고 있습니다. 반려동물 제품의 고급화로 인해 제품의 세분화와 맞춤형 니즈가 증가하고 있으며, 이는 펫케어 산업의 성장을 이끌고 있습니다. 시장조사에 따르면 2023년 전세계 펫 케어 시장의 규모는 약 1천 844억이며, 인도 펫푸드 시장은 주요 대기업들이 장악하고 있으며 신제품 개발과 사업 영역 확장에 적극적으로 투자하고 있습니다. 또한, 반려동물 친화 공간이 늘어나고 반려동물 관련 이벤트가 인기를 끌고 있으며, 인도 펫케어 산업은 프리미엄화 추세를 보이고 있습니다. 이러한 동향들을 바탕으로 인도 펫케어 식품 시장은 계속해서 성장할 것으로 예상됩니다.


In [29]:
# Chain 실행
rag_chain = set_chain(our_llm, sq_retriever, qa_prompt2)
question = "인도 펫케어 식품 시장 동향을 알려주세요."
response = rag_chain.invoke({'input':question})
print(response['answer'])

```json
{
    "query": "인도 펫케어 식품 시장 동향",
    "filter": "NO_FILTER"
}
```인도 펫케어 식품 시장은 현재 지속적인 성장을 보이고 있습니다. 반려동물 제품의 고급화로 인해 제품의 세분화와 맞춤형 니즈가 증가하고 있으며, 이는 시장의 다양성을 증가시키고 있습니다. 유로모니터에 따르면 2023년 전세계 펫 케어 시장의 규모는 약 1천 844억이며, 이 중 인도 펫케어 시장도 지속적인 성장을 이어가고 있습니다.

또한, 인도 펫푸드 시장은 주요 대기업들이 시장을 장악하고 있으며, 신제품 개발에 적극적으로 투자하고 있습니다. 대기업들은 인수 및 확장을 통해 사업 영역을 넓히고, 연구와 개발에 집중하여 보다 저렴한 가격의 새로운 제품을 소비자에게 제공하고 있습니다. 또한, Mankind Pharma와 같은 유명 제약 회사도 펫푸드 시장에 진출하여 시장 경쟁을 더욱 촉진하고 있습니다.

인도 펫케어 산업은 프리미엄화 추세를 보이고 있지만, 일부 소비자들은 인플레이션 상승과 가처분소득으로 중저가 제품을 선호하는 경향이 있습니다. 그러나 반려동물 입양이 계속해서 증가하고 있기 때문에 사료 시장 전반은 꾸준한 성장이 예상되고 있습니다. 또한, 단백질과 비타민이 풍부한 펫푸드가 시장에 출시되어 소비자들의 다양한 니즈를 충족시키고 있습니다.

이러한 동향들을 종합하면, 인도 펫케어 식품 시장은 성장 가능성이 높고 다양한 기회를 제공하고 있습니다.


In [30]:
# Chain 실행
rag_chain = set_chain(our_llm, pd_retriever, qa_prompt2)
question = "인도 펫케어 식품 시장 동향을 알려주세요."
response = rag_chain.invoke({'input':question})
print(response['answer'])

인도의 펫케어 식품 시장은 지속적으로 성장하고 있습니다. 인도의 경제적 성장과 함께 반려동물을 키우는 가구 수가 증가하고 있어서 펫케어 시장이 확대되고 있습니다. 특히, 고급 제품에 대한 수요가 증가하고 있으며, 건강식품 및 영양 보충제에 대한 관심도 높아지고 있습니다. 또한, 온라인 판매 채널을 통한 펫케어 제품 구매가 증가하고 있으며, 이는 소비자들이 더 많은 옵션을 찾고 편리성을 추구하는 추세에 부합합니다. 이러한 동향들을 고려하면 인도의 펫케어 식품 시장은 미래에도 계속해서 성장할 것으로 전망됩니다.


In [22]:
# Chain 실행
rag_chain = set_chain(our_llm, web_retriever, qa_prompt2)
question = "인도 펫케어 식품 시장 동향을 알려주세요."
response = rag_chain.invoke({'input':question})
print(response['answer'])

1. 인도 펫케어 식품 시장의 최신 동향은 무엇인가요?
2. 인도의 펫케어 식품 시장은 어떻게 변화하고 있나요?
3. 인도에서의 펫케어 식품 시장 동향을 알려주세요.

Fetching pages: 100%|##########| 2/2 [00:00<00:00,  2.12it/s]


인도의 펫케어 식품 시장은 현재 지속적인 성장을 보이고 있습니다. 2023년에는 전 세계 펫케어 시장 규모가 약 1천 844억 달러인 가운데, 인도의 펫케어 시장 규모는 약 6억 달러로 집계되었습니다. 이는 2018년부터 2023년까지 연평균 14-15%로 성장하고 있는 것으로 나타났습니다. 특히, 코로나19 이후에는 급격한 성장을 보여 두 자리 수를 유지하고 있습니다. 또한, 2028년까지는 약 11억 달러 규모로 성장할 것으로 예측되고 있습니다.

인도의 펫케어 시장에서는 펫휴머니제이션(Pet Humanisation)이 보편화되고 있어, 반려동물을 가족의 일원으로 취급하는 경향이 강조되고 있습니다. 특히 대도시 중산층 이상 가정에서는 반려동물의 생활 환경과 건강에 대한 관심이 높아지고 있으며, 이에 따라 화학첨가물이 없는 유기농 사료나 영양을 증진시킬 수 있는 제품을 선호하는 추세가 있습니다.

또한, 코로나19 이후에는 반려동물의 웰니스를 위한 시설의 수요가 증가하고 있습니다. 대도시에서는 반려동물을 동반하여 여행할 수 있는 규정이 확대되고 있으며, 반려동물 친화적인 공간이 늘어나고 있습니다. 이러한 환경 변화와 Z세대의 활발한 소셜미디어 활동 등의 영향으로 인도의 펫케어 시장은 계속해서 성장할 것으로 예상됩니다.


### 만들어 논 평가 데이터셋 불러오기 

In [13]:
path = '../my_data/qa_new.parquet'

# make_qa.py 파일로 만듬 : 질문과 응답 쌍을 만듬 , 어떤 문서가 관련이 있는지 
df = pd.read_parquet(path)

df

,qid,retrieval_gt,query,generation_gt
0,e28e176e-2db2-46f6-a360-74dfffe0377f,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인...,자료에 나와 있는 날짜는 언제인가요?,[자료에 나와 있는 날짜는 2016년 8월 24일과 2016년 12월 30일입니다.]
1,29aad4ed-e426-4100-a204-c11d17197100,[[PDF_without_contents\2. 경제 및 시장 분석\[전체 정보][K...,인도에서 채식주의자의 인구비율이 높은 이유는 무엇인가요?,[인도는 종교적 특성으로 인해 채식주의자의 인구비율이 높게 나타납니다.]
2,54f2b135-b0bd-434e-9e92-ca29798988c3,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코...,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,[상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 ...
3,f3a4828f-9954-4c08-ba33-f81dfa3ce819,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인...,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,[인증서의 정지 및 취소는 Schedule II에 명시된 적용 가능한 스킴에 따라 ...
4,f78f2c4f-687d-4547-add3-5b632bfaa08d,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인...,불만 사항의 해결 조치는 몇 일 내에 완료되어야 하나요?,[불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완...
...,...,...,...,...
73,6205c3ba-a01d-45d1-a0ba-1552dc2026f2,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인...,월별 생산 실적을 제공해야 하는 기간은 언제인가요?,[해당 자료와 관련이 없습니다.]
74,e505c9a3-6128-404f-bc59-9819f81b2cf6,[[PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코...,인도에서 토지 취득 시 발생할 수 있는 주요 문제는 무엇인가요?,"[인도에서 토지 취득 시 법적 소유권을 확립하기 어렵고, 상속 분할, 지분 매도인의..."
75,38a4b67d-812e-45c5-8703-f552277af964,[[PDF_without_contents\2. 경제 및 시장 분석\[전체 정보][K...,인도에서 연락사무소를 설치하고 운영하기 위해 필요한 법률은 무엇인가요?,[인도에서 연락사무소를 설치하고 운영하기 위해 필요한 법률은 외국환관리법(Forei...
76,2d69c2a2-5440-4645-a4fa-1b8906f8ac1a,[[PDF_without_contents\2. 경제 및 시장 분석\펫산업\[인도_펫...,Mars는 건강한 스낵을 생산하는 데 있어서 주도적인 역할을 하고 있나요?,"[아니요, Mars는 건강한 스낵을 생산하는 데 있어서 주도적인 역할을 하지 않는 ..."


In [14]:
df['retrieval_gt'] = df['retrieval_gt'].apply(lambda x: x[0][0])
df['generation_gt'] = df['generation_gt'].apply(lambda x: x[0])

In [15]:
df.head()

,qid,retrieval_gt,query,generation_gt
0,e28e176e-2db2-46f6-a360-74dfffe0377f,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,자료에 나와 있는 날짜는 언제인가요?,자료에 나와 있는 날짜는 2016년 8월 24일과 2016년 12월 30일입니다.
1,29aad4ed-e426-4100-a204-c11d17197100,PDF_without_contents\2. 경제 및 시장 분석\[전체 정보][KOT...,인도에서 채식주의자의 인구비율이 높은 이유는 무엇인가요?,인도는 종교적 특성으로 인해 채식주의자의 인구비율이 높게 나타납니다.
2,54f2b135-b0bd-434e-9e92-ca29798988c3,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코트라...,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 사...
3,f3a4828f-9954-4c08-ba33-f81dfa3ce819,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,인증서의 정지 및 취소는 Schedule II에 명시된 적용 가능한 스킴에 따라 이...
4,f78f2c4f-687d-4547-add3-5b632bfaa08d,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,불만 사항의 해결 조치는 몇 일 내에 완료되어야 하나요?,불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완료...


In [26]:
# df.to_excel('../my_data/qa_new.xlsx', index=False)

In [16]:
qa_data = pd.read_excel('../my_data/qa_new.xlsx')

In [17]:
qa_data.head()

,qid,retrieval_gt,query,generation_gt
0,e28e176e-2db2-46f6-a360-74dfffe0377f,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,자료에 나와 있는 날짜는 언제인가요?,자료에 나와 있는 날짜는 2016년 8월 24일과 2016년 12월 30일입니다.
1,29aad4ed-e426-4100-a204-c11d17197100,PDF_without_contents\2. 경제 및 시장 분석\[전체 정보][KOT...,인도에서 채식주의자의 인구비율이 높은 이유는 무엇인가요?,인도는 종교적 특성으로 인해 채식주의자의 인구비율이 높게 나타납니다.
2,54f2b135-b0bd-434e-9e92-ca29798988c3,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][코트라...,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 사...
3,f3a4828f-9954-4c08-ba33-f81dfa3ce819,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,인증서의 정지 및 취소는 Schedule II에 명시된 적용 가능한 스킴에 따라 이...
4,f78f2c4f-687d-4547-add3-5b632bfaa08d,PDF_without_contents\1. 법률 및 규제\[법률_규범_특허][인도표...,불만 사항의 해결 조치는 몇 일 내에 완료되어야 하나요?,불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완료...


In [18]:
qa_data = qa_data[['query', 'generation_gt']]
qa_data.columns = ['question', 'ground_truth']

qa_data.head()

,question,ground_truth
0,자료에 나와 있는 날짜는 언제인가요?,자료에 나와 있는 날짜는 2016년 8월 24일과 2016년 12월 30일입니다.
1,인도에서 채식주의자의 인구비율이 높은 이유는 무엇인가요?,인도는 종교적 특성으로 인해 채식주의자의 인구비율이 높게 나타납니다.
2,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 사...
3,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,인증서의 정지 및 취소는 Schedule II에 명시된 적용 가능한 스킴에 따라 이...
4,불만 사항의 해결 조치는 몇 일 내에 완료되어야 하나요?,불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완료...


In [20]:
rag_chain = set_chain(our_llm, retriever, qa_prompt2)

In [21]:

qa_data['answer'] = qa_data['question'].apply(lambda x: rag_chain.invoke({'input':x})['answer'])
qa_data['contexts'] = qa_data['question'].apply(lambda x: [d.page_content for d in retriever.get_relevant_documents(x)])
# 5분 걸림
qa_data.head()

,question,ground_truth,answer,contexts
0,자료에 나와 있는 날짜는 언제인가요?,자료에 나와 있는 날짜는 2016년 8월 24일과 2016년 12월 30일입니다.,제공된 자료에는 다양한 날짜가 나와 있습니다. \n\n- 주요 정치 일정:\n -...,[월별 1월 2월 3월 4월 5월 6월 7월 8월 9월 10월 11월 12월\n수 ...
1,인도에서 채식주의자의 인구비율이 높은 이유는 무엇인가요?,인도는 종교적 특성으로 인해 채식주의자의 인구비율이 높게 나타납니다.,인도에서 채식주의자의 인구비율이 높은 이유는 종교적 특성과 문화적 요인이 크게 작용...,[이 높게 나타나며 종교적 특성으로 인해 자연제품을 선호하는 경향이 있다\n 채식주...
2,상표 등록이 완료된 후 기호 을을 사용할 수 있는 조건은 무엇인가요?,상표 등록이 완료된 후에는 인증서에 명시된 상품 및 서비스에 대해서만 기호 을을 사...,"상표 등록이 완료된 후 기호 ""을""을 사용할 수 있는 조건은 등록된 상표가 소유자의...",[원이 완료되면 기호 을 제품과 함께 사용할 수 있으며 상표 등록이 완료 후에 기호...
3,인증서의 정지 및 취소는 어떤 절차에 따라 이루어지나요?,인증서의 정지 및 취소는 Schedule II에 명시된 적용 가능한 스킴에 따라 이...,인증서의 정지 및 취소는 일반적으로 다음과 같은 절차에 따라 이루어집니다:\n\n1...,[있다 wwwsafescryptcom wwwnicin wwwidrbtcaorgin ...
4,불만 사항의 해결 조치는 몇 일 내에 완료되어야 하나요?,불만 사항의 해결 조치는 제품 테스트 시간이 포함되지 않은 경우 90일 이내에 완료...,불만 사항의 해결 조치는 일반적으로 다양한 요인에 따라 다를 수 있습니다. 제공된 ...,[채무변제 능력에 대한 이사의 결의에는 다음의 서류가 첨부되어야 한다\ncid0 직...


In [37]:
# qa_data.to_excel('../my_data/exper_data.xlsx')

In [ ]:
qa_data = pd.read_excel('../my_data/qa_new.xlsx')

In [22]:
from datasets import Dataset 

dataset = Dataset.from_pandas(qa_data)

dataset[0]

{'question': '자료에 나와 있는 날짜는 언제인가요?',
 'ground_truth': '자료에 나와 있는 날짜는 2016년 8월 24일과 2016년 12월 30일입니다.',
 'answer': '제공된 자료에는 다양한 날짜가 나와 있습니다. \n\n- 주요 정치 일정:\n  - 헌법 제정일 (Republic Day): 1월 26일\n  - 독립기념일 (Independence Day): 8월 15일\n  - 인도 총선: 5월 2024\n\n- 주요 경제·통상 일정:\n  - 인도 2024/25 회계연도 연방예산안 발표: 2월 1일\n\n- 유망 전시회 캘린더:\n  - 인도 의료기기 엑스포: 1월 5∼7일\n  - 인도 첸나이 글로벌 투자자 미팅 투자 전시회: 1월 7∼8일\n  - 바이브런트 구자라트 서밋: 1월 10∼12일\n  - 컨버전스 인디아 엑스포: 1월 17∼19일\n  - 인도 홈/퍼스널 케어 전시회: 1월 18∼19일\n  - 인도 문구/사무용품 전시회: 1월 18∼20일\n  - 인도 기프트쇼: 1월 18∼20일\n  - 인도 섬유 전시회: 1월 18∼20일\n\n따라서, 제공된 자료에 나와 있는 날짜는 1월 5일부터 8월 15일까지 다양하게 포함되어 있습니다.',
 'contexts': ['월별 1월 2월 3월 4월 5월 6월 7월 8월 9월 10월 11월 12월\n수 출 1466 1351 1508 1542 1702 1429 2361 1631 1615 1403 1380 1490\n증감 153 51 129 141 712 225 920 268 81 02 63 134\n수 입 695 732 981 891 1078 856 848 682 579 522 537 498\n증감 240 569 548 214 849 460 192 75 244 296 321 331',
  '무시간은 통상 09301700 점심시간은 13001400 라는 점을 유의하여 미팅 시간을 잡는 것이 좋다 인도를 방문할 시 우기79월\n에는 많은 비가 내려 이동이 쉽지 않기 때문

In [23]:
from ragas.metrics import faithfulness, answer_relevancy, context_recall, context_precision

In [27]:
from ragas import evaluate 
from langchain_community.chat_models import ChatOllama 
from langchain_community.embeddings.ollama import OllamaEmbeddings

langchain_llm = ChatOllama(model="gemma2")
langchain_embeddings = OllamaEmbeddings(model="gemma2")

result = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision
    ],
    llm=langchain_llm,
    embeddings=langchain_embeddings,
    raise_exceptions=False
)


result

AttributeError: 'RunnableSequence' object has no attribute 'set_run_config'

In [42]:
df1 = pd.DataFrame(list(result.items()), columns=['Metric', 'Gemma2'])

df1.head()


,Metric,Gemma2
0,faithfulness,0.785026
1,answer_relevancy,0.351454
2,context_recall,0.514706
3,context_precision,0.517290


In [43]:
df1.to_excel('../experiments/chat_gemma2.xlsx')

In [24]:
rag_chain = set_chain(our_llm, retriever, qa_prompt2)

In [26]:
from ragas import evaluate 
from langchain_community.chat_models import ChatOllama 
from langchain_community.embeddings.ollama import OllamaEmbeddings
from utils.config import config

# langchain_llm = ChatOllama(model="gemma2")
langchain_embeddings = OllamaEmbeddings(model="gemma2")
# langchain_embeddings = OpenAIEmbeddings(model=config['embed_model']['model_name'])

result2 = evaluate(
    dataset,
    metrics = [
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision
    ],
    llm=rag_chain,
    embeddings=langchain_embeddings,
    raise_exceptions=False
)


result2

AttributeError: 'RunnableSequence' object has no attribute 'set_run_config'